In [ ]:
# Konlpy 설치
!apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!apt-get install python3-dev; pip3 install konlpy # Python 3.x

# Kiwi 형태소분석기 설치
!pip install kiwipiepy

# colab에서 한글폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# PDF reader 설치
!pip install PyPDF2

In [ ]:
!find / -name 'Nanum*'

/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf
/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf
/usr/share/fonts/truetype/nanum/NanumGothic.ttf
/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf
/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf
/usr/share/fonts/truetype/nanum/NanumSquareB.ttf
/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf
/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf
/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf
/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf
/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf
/usr/share/fonts/truetype/nanum/NanumSquareR.ttf
find: ‘/proc/56/task/56/net’: Invalid argument
find: ‘/proc/56/net’: Invalid argument


## 1. 1. 필수 라이브러리 임포트

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

from PyPDF2 import PdfReader # pdf를 텍스트로 읽어오는 라이브러리
from wordcloud import WordCloud

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from PIL import Image

import re #정규표현식
from konlpy.tag import Okt # 한국어 형태소분석을 위한 라이브러리
okt = Okt()

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
kiwi = Kiwi()

## 1. 2. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  PDF 파일
data_path = '/content/drive/MyDrive/박완서/6권 그 여자네 집_교정교열완료/6권 그여자네집 합본.pdf'

In [ ]:
reader = PdfReader(data_path)
pages = reader.pages

In [ ]:
raw_docs = ''
for page in pages:
    raw_docs += page.extract_text()

# Step 2. 워드클라우드 생성

## 2. 1. 데이터 정제
    - 정규표현식을 활용한 특정문자열 패턴제거 및 수정
    - 단어 필터링
        - 품사태깅 및 필터링
        - 불용어리스트 활용 필터링

In [ ]:
# 단어 및 일반 띄어쓰기가 아닌 문자열 모두 제거
is_not_word = re.compile('[^ㄱ-ㅎ가-힣a-zA-Z ]') #특정문자만 인식, 제거하는 정규표현식
documents = is_not_word.sub('', raw_docs)
documents

In [ ]:
# 강제개행 및 위 작업이후 생긴 2개 이상의 띄어쓰기 하나로 압축하기
is_double_space = re.compile(' +')
documents = is_double_space.sub(' ', documents)
documents

In [ ]:
#stopwords 방식1.  파일 불러와서 입력
with open('/content/drive/MyDrive/데이터/Colab Notebooks/불용어파일.txt', 'r') as f:
    stopwords=f.read().splitlines()

In [ ]:
stopwords


In [ ]:
filtered_words = []

In [ ]:
kiwi.tokenize(documents)

In [ ]:
# 단어 토큰화 - Kiwi 버전
# Kiwi 토크나이저 참조 : https://github.com/bab2min/kiwipiepy
tokenized_docs = kiwi.tokenize(documents)
for token in tokenized_docs:
    tag, word = token.tag, token.form
    if tag in ['NNG', 'NNP'] and word not in stopwords and len(word)>=2:
        filtered_words.append(word)

In [ ]:
filtered_words

In [ ]:
# 단어빈도 딕셔너리 생성
word_freq = dict(Counter(filtered_words))
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# 가장 많이 등장한 단어 확인
sorted_word_freq[:30]  #숫자만 조절

## 2. 2. 워드클라우드
    - 기본 워드클라우드
    - Masked 워드클라우드

In [ ]:
font_path = '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf'
wc = WordCloud(font_path = font_path,
               background_color = 'white',
               width = 1600,
               height = 1600).generate_from_frequencies(word_freq)

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()